In [1]:
from gurobipy import *
import snap
from graphviz import Digraph
import pandas as pd
import numpy as np

In [2]:
# G1 = snap.TUNGraph.New()
# G1.AddNode(0)
# G1.AddNode(1)
# G1.AddNode(2)
# G1.AddNode(3)
# G1.AddNode(4)
# G1.AddNode(5)
# G1.AddNode(6)
# G1.AddNode(7)
# G1.AddNode(8)
# G1.AddNode(9)
# G1.AddNode(10)
# G1.AddNode(11)
# G1.AddNode(12)
# G1.AddNode(13)
# G1.AddNode(14)
# G1.AddEdge(0,1)
# G1.AddEdge(0,2)
# G1.AddEdge(2,3)
# G1.AddEdge(1,3)
# G1.AddEdge(3,4)
# G1.AddEdge(4,5)
# G1.AddEdge(4,6)
# G1.AddEdge(0,7)
# G1.AddEdge(7,8)
# G1.AddEdge(7,9)
# G1.AddEdge(9,10)
# G1.AddEdge(0,11)
# G1.AddEdge(11,12)
# G1.AddEdge(11,13)
# G1.AddEdge(13,14)
# G1

In [3]:
# G1 = snap.GenRndGnm(snap.TUNGraph, 20, 50)
# G1 = snap.LoadEdgeList(snap.TUNGraph, "LastFM.txt", 0, 1)
#匯入圖
G1 = snap.LoadEdgeList(snap.TUNGraph, "graph.txt", 0, 1)

In [4]:
#刪除孤單的點
lonely_node = []
for NI in G1.Nodes():
    if NI.GetOutDeg()==0:
        lonely_node.append(NI.GetId())
        G1.DelNode(NI.GetId())

print("lonely_node = ",lonely_node)

lonely_node =  []


In [5]:
S = 1 #初始感染點(有四個要改)
AllnodeID = []
InfectednodeID = [S]
V = [] #AllnodeID\0(s) #vulnerablenodeID
vaccinatednodeID = [] #vaccinated的node
for NI in G1.Nodes():
    AllnodeID.append(NI.GetId())
    if(NI.GetId()!=S):
        V.append(NI.GetId())

In [6]:
# AllnodeID = [] #全部的node
# InfectednodeID = [0]
# vulnerablenodeID = []
# vaccinatednodeID = [] #vaccinated的node
# # inf_plus_vul_minus_zero = []
# for NI in G1.Nodes():
#     AllnodeID.append(NI.GetId())
#     if(NI.GetId()!=0):
#         vulnerablenodeID.append(NI.GetId())
#     print("node: %d" % NI.GetId())

In [7]:
layer = 0
for NI in G1.Nodes():
    if(G1.GetShortPath(NI.GetId(),S)>layer):
        layer = G1.GetShortPath(NI.GetId(),S) 
layer

5

# IP Programming

In [8]:
B = 1 #budget
S = 1 #初始感染點
#Create empty model
IP_model = Model('IP Vaccination Strategy')

#add variablies
#S = model.addVars([(u,t) for u in V for t in range(1,layer+1)],vtype=GRB.BINARY, name="S")
X = IP_model.addVars([(u,t) for u in V for t in range(1,layer+1) ],vtype=GRB.BINARY, name="X")
Y = IP_model.addVars([(v) for v in V],vtype=GRB.BINARY, name="Y")
IP_model.update()
#add objective function
# obj = 0
# for v in V:
#     obj = obj+Y[v]
IP_model.setObjective(quicksum(Y[v] for v in V) ,GRB.MAXIMIZE)

#constraint 1
IP_model.addConstrs(quicksum(X[u,t] for u in V)<=B for t in range(1,layer+1))
#constraint 2
#model.addConstrs(quicksum(X[u,t] for t in range(1,layer+1))<=1 for u in V) 
#constraint 3
IP_model.addConstrs(quicksum(X[u,t] for u in V for t in range(1,(G1.GetShortPath(S,v)-G1.GetShortPath(u,v))+1) if t>=0 )>= Y[v]  for v in V)
IP_model.update()
IP_model.optimize()

Academic license - for non-commercial use only - expires 2021-07-24
Using license file C:\Users\user\gurobi.lic


KeyboardInterrupt: 

In [ ]:
print("IP Programming")
print("保護的個數=",IP_model.ObjVal)
for var in IP_model.getVars():
    if(var.x>0):
        print(var.varName,'\t', var.x)

# LP Programming

In [ ]:
B = 1 #budget
S = 1 #初始感染點
#Create empty model
LP_model = Model('LP Vaccination Strategy')

#add variablies
#S = model.addVars([(u,t) for u in V for t in range(1,layer+1)],vtype=GRB.BINARY, name="S")
X = LP_model.addVars([(u,t) for u in V for t in range(1,layer+1) ],lb=0,ub=1, name="X")
Y = LP_model.addVars([(v) for v in V],lb=0,ub=1, name="Y")
LP_model.update()
#add objective function
# obj = 0
# for v in V:
#     obj = obj+Y[v]
LP_model.setObjective(quicksum(Y[v] for v in V) ,GRB.MAXIMIZE)

#constraint 1
LP_model.addConstrs(quicksum(X[u,t] for u in V)<=B for t in range(1,layer+1))
#constraint 3
LP_model.addConstrs(quicksum(X[u,t] for u in V for t in range(1,(G1.GetShortPath(S,v)-G1.GetShortPath(u,v))+1) if t>=0 )>= Y[v]  for v in V)
LP_model.update()
LP_model.optimize()

In [ ]:
THE_X = [] #取出X[,]
X_Pro = [] #取出X[,]的機率
print("LP Programming")
print("保護的個數=",LP_model.ObjVal)
for var in LP_model.getVars():
    if(var.x>0):
        if(var.varName[0]=="X"):
            THE_X.append(var.varName) 
            X_Pro.append(var.x)
        print(var.varName,'\t', var.x)

# LP Rounding

In [ ]:
def copy_graph(graph):
    tmpfile = '.copy.bin'

    # Saving to tmp file
    FOut = snap.TFOut(tmpfile)
    graph.Save(FOut)
    FOut.Flush()

    # Loading to new graph
    FIn = snap.TFIn(tmpfile)
    graphtype = type(graph)
    new_graph = graphtype.New()
    new_graph = new_graph.Load(FIn)

    return new_graph

In [ ]:
# import random
# print("LP Rounding")
# for i in range(50):
#     print("round=",i)
#     random_strategy_choose = [] #新策略(格式很亂)
#     new_strategy = []#新策略(格式清理過)
#     for i in range(1,layer+1): #以每一個時間抓出來算機率
#         XX = [] 
#         XX_Pro = []

#         for x in range(len(THE_X)):
#             if THE_X[x][-2]==str(i):
#                 XX.append(THE_X[x])
#                 XX_Pro.append(X_Pro[x])
#         rdc = random.choices(XX, weights=XX_Pro) #這次的策略
#         random_strategy_choose.append(rdc)
# #         print( "選擇的是:",rdc )
#     for i in range(len(random_strategy_choose)):#清理格式
#         new_strategy.append(random_strategy_choose[i][0][2:-3])

#     new_strategy = list(map(int, new_strategy)) #str-->int
#     print("選擇的策略=",new_strategy)
#     ########################################################################開始注射策略
#     G_temp = copy_graph(G1)
#     S = 0 #初始感染點
#     temp_vaccinated = []
#     temp_infected = [0]
#     temp_vulnerable = V.copy()    
#     for i in range(len(new_strategy)): #開始依序注射
#         temp_saved = []
#         b = temp_vulnerable.copy()
#         if(new_strategy[i] not in temp_vulnerable): #如果要注射的點已經被保護了(從圖中刪除)
#             for n in range(len(b)):
#                  if(G_temp.GetShortPath(S,b[n])==i+1):#病毒開始擴散
#                     temp_infected.append(b[n])
#                     temp_vulnerable.remove(b[n])

#             continue

#         for k in range(len(temp_vulnerable)):#每一個點計算是否有被注射的點保護

#             r  = i+1

#             if(r<=(G_temp.GetShortPath(S,temp_vulnerable[k])-G_temp.GetShortPath(new_strategy[i],temp_vulnerable[k]))):
#                 temp_saved.append(temp_vulnerable[k])

#         for m in range(len(temp_saved)): #調整位置
#             temp_vaccinated.append(temp_saved[m])
#             temp_vulnerable.remove(temp_saved[m])
#             G_temp.DelNode(temp_saved[m])

#         a = temp_vulnerable.copy()
#         for n in range(len(a)):#病毒開始擴散
#             if(G_temp.GetShortPath(S,a[n])==i+1):
#                 temp_infected.append(a[n])
#                 temp_vulnerable.remove(a[n])

            
#     print("保護個數=",len(temp_vaccinated))
# #     print("被保護的點=",temp_vaccinated)
            

In [ ]:
import random
timer = 10
count = 0
print("LP Rounding")
for i in range(timer):
    print("round=",i)
    random_strategy_choose = [] #新策略(格式很亂)
    new_strategy = []#新策略(格式清理過)
    for i in range(1,layer+1): #以每一個時間抓出來算機率
        XX = [] 
        XX_Pro = []

        for x in range(len(THE_X)):
            if THE_X[x][-4]==",":
                if THE_X[x][-3:-1]==str(i):
                    XX.append(THE_X[x])
                    XX_Pro.append(X_Pro[x])
            elif THE_X[x][-2]==str(i):
                XX.append(THE_X[x])
                XX_Pro.append(X_Pro[x])
        rdc = random.choices(XX, weights=XX_Pro) #這次的策略
        random_strategy_choose.append(rdc)
#         print( "選擇的是:",rdc )
    for i in range(len(random_strategy_choose)):#清理格式
        if(random_strategy_choose[i][0][-4]==","):
            new_strategy.append(random_strategy_choose[i][0][2:-4])
        else:
            new_strategy.append(random_strategy_choose[i][0][2:-3])

    new_strategy = list(map(int, new_strategy)) #str-->int
    print("選擇的策略=",new_strategy)
    ########################################################################開始注射策略
    G_temp = copy_graph(G1)
    S = 1 #初始感染點
    temp_vaccinated = []
    temp_infected = [S]
    temp_vulnerable = V.copy()    
    for i in range(len(new_strategy)): #開始依序注射
        temp_saved = []
        b = temp_vulnerable.copy()
        if(new_strategy[i] not in temp_vulnerable): #如果要注射的點已經被保護了(從圖中刪除)
            for n in range(len(b)):
                 if(G_temp.GetShortPath(S,b[n])==i+1):#病毒開始擴散
                    temp_infected.append(b[n])
                    temp_vulnerable.remove(b[n])

            continue

        for k in range(len(temp_vulnerable)):#每一個點計算是否有被注射的點保護

            r  = i+1

            if(r<=(G_temp.GetShortPath(S,temp_vulnerable[k])-G_temp.GetShortPath(new_strategy[i],temp_vulnerable[k]))):
                temp_saved.append(temp_vulnerable[k])

        for m in range(len(temp_saved)): #調整位置
            temp_vaccinated.append(temp_saved[m])
            temp_vulnerable.remove(temp_saved[m])
            G_temp.DelNode(temp_saved[m])

        a = temp_vulnerable.copy()
        for n in range(len(a)):#病毒開始擴散
            if(G_temp.GetShortPath(S,a[n])==i+1):
                temp_infected.append(a[n])
                temp_vulnerable.remove(a[n])

            
    print("保護個數=",len(temp_vaccinated))
    count = count+int(len(temp_vaccinated))
#     print("被保護的點=",temp_vaccinated)

print("平均被保護個數=",count/timer)